In [1]:
from bs4 import BeautifulSoup
from typing import List, Dict
from datetime import datetime
import requests
import pandas as pd
import json
import time
import os 

In [3]:
def get_news_list(page: int) -> str:
    """AI타임즈 뉴스 목록 페이지의 HTML을 가져오는 함수"""
    url = "https://www.aitimes.com/news/articleList.html"
    params = {
        "page": page,
        "view_type": "sm"
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/120.0.0.0'
    }
    
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"페이지 {page} 요청 중 에러 발생: {e}")
        return ""

def get_article_content(url: str) -> str:
    """기사 상세 페이지의 내용을 가져오는 함수"""
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/120.0.0.0'
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 기사 본문 찾기
        article_content_div = soup.find("article", id="article-view-content-div")
        if article_content_div:
            # 모든 p 태그 찾기
            paragraphs = article_content_div.find_all("p")
            # 각 단락의 텍스트를 리스트로 모으기
            content_list = [p.text.strip() for p in paragraphs if p.text.strip()]
            # 단락들을 개행문자로 결합
            return '\n'.join(content_list)
        return ""
    except requests.exceptions.RequestException as e:
        print(f"기사 내용 요청 중 에러 발생: {e}")
        return ""

def parse_news_info(html: str) -> List[Dict]:
    """HTML에서 뉴스 기사 정보를 파싱하는 함수"""
    articles = []
    soup = BeautifulSoup(html, 'html.parser')
    
    content_list = soup.find("section", id="section-list")
    if not content_list:
        return articles
        
    news_items = content_list.find_all("li")
    print(f"찾은 {len(news_items)}개의 기사")
    
    for item in news_items:
        # 제목 추출
        title_elem = item.find("h4", class_="titles")
        title = title_elem.find("a").text.strip() if title_elem else ""
        
        # URL 추출
        url_elem = title_elem.find("a") if title_elem else None
        url = "https://www.aitimes.com" + url_elem["href"] if url_elem and "href" in url_elem.attrs else ""
        
        # 설명 추출
        desc_elem = item.find("p", class_="lead")
        description = desc_elem.find("a").text.strip() if desc_elem else ""
        
        # 날짜 추출
        date_elem = item.find("span", class_="byline")
        if date_elem:
            date = date_elem.find_all("em")[-1].text.strip()
        else:
            date = ""
            
        # 기사 본문 가져오기
        content = get_article_content(url) if url else ""
        
        article = {
            "title": title,
            "description": description,
            "url": url,
            "date": date,
            "content": content
        }
        
        articles.append(article)
        # 서버 부하를 줄이기 위한 지연
        time.sleep(1)
    
    return articles

def get_date_range(df: pd.DataFrame) -> tuple:
    """데이터프레임에서 가장 빠른 날짜와 가장 늦은 날짜를 추출하는 함수"""
    try:
        # 날짜 문자열을 datetime 객체로 변환
        df['parsed_date'] = pd.to_datetime(df['date'], format='%Y.%m.%d %H:%M')
        
        # 가장 빠른 날짜와 가장 늦은 날짜 추출
        start_date = df['parsed_date'].min().strftime('%Y%m%d')
        end_date = df['parsed_date'].max().strftime('%Y%m%d')
        
        # 임시 컬럼 삭제
        df.drop('parsed_date', axis=1, inplace=True)
        
        return start_date, end_date
    except Exception as e:
        print(f"날짜 처리 중 에러 발생: {e}")
        return None, None

def save_to_files(articles: List[Dict], start_page: int, end_page: int, base_path: str = "./files"):
    """뉴스 기사 정보를 DataFrame으로 변환하여 CSV와 JSON 파일로 저장하는 함수"""
    # DataFrame 생성
    df = pd.DataFrame(articles)
    
    # 날짜 범위 추출
    start_date, end_date = get_date_range(df)
    
    if not (start_date and end_date):
        # 날짜 추출 실패시 현재 시간 사용
        current_time = datetime.now().strftime("%Y%m%d_%H%M")
        file_name = f"ai_times_news_p{start_page}-{end_page}_{current_time}.json"
    else:
        # 날짜 범위와 페이지 정보를 포함한 파일명 생성
        file_name = f"ai_times_news_{start_date}-{end_date}_p{start_page}-{end_page}.json"
    
    
    # DataFrame을 JSON 파일로 저장
    json_path = os.path.join(base_path, file_name)
    df.to_json(json_path, force_ascii=False, orient='records', indent=4)
    print(f"JSON 파일 저장 완료: {json_path}")
    
    # 데이터 수집 정보 출력
    print("\n[데이터 수집 정보]")
    print(f"수집 페이지: {start_page}-{end_page} 페이지")
    print(f"수집 기간: {start_date[:4]}.{start_date[4:6]}.{start_date[6:]} - {end_date[:4]}.{end_date[4:6]}.{end_date[6:]}")
    print(f"수집 기사: 총 {len(df)}건")
    
    # 데이터 미리보기
    print("\n[데이터 미리보기]")
    print(df.head())


In [ ]:
def main():
    start_page = 1
    end_page = 1  # 수집할 마지막 페이지 번호
    page_unit = 10  # 한 번에 처리할 페이지 수
    download_folder = './files/ai_news' #저장 폴더 이름
    os.makedirs(download_folder, exist_ok=True) #폴더 생성
    print("뉴스 기사 정보 수집 중...")
    
    # 10페이지씩 처리
    for start_idx in range(start_page, end_page + 1, page_unit):
        all_articles = []
        end_idx = min(start_idx + page_unit - 1, end_page)
        
        print(f"\n=== {start_idx}~{end_idx} 페이지 수집 시작 ===")
        
        for page in range(start_idx, end_idx + 1):
            html = get_news_list(page)
            if html:
                page_articles = parse_news_info(html)
                all_articles.extend(page_articles)
                print(f"페이지 {page}: {len(page_articles)}개의 기사 정보 수집 완료")
        
        if not all_articles:
            print(f"{start_idx}~{end_idx} 페이지의 기사 정보를 가져오는데 실패했습니다.")
            continue
        
        print(f"\n{start_idx}~{end_idx} 페이지 : 총 {len(all_articles)}개의 기사 정보를 수집했습니다.")
        
        try:
            save_to_files(all_articles, start_idx, end_idx, download_folder)
        except Exception as e:
            print(f"데이터 저장 중 에러 발생: {e}")
        
        print(f"=== {start_idx}~{end_idx} 페이지 처리 완료 ===\n")
        
        # 다음 수집 전 잠시 대기
        time.sleep(0.1)

    print("\n모든 페이지 수집 완료!")


In [5]:
if __name__ == "__main__":
    main()

뉴스 기사 정보 수집 중...

=== 1~1 페이지 수집 시작 ===
찾은 20개의 기사
페이지 1: 20개의 기사 정보 수집 완료

1~1 페이지 : 총 20개의 기사 정보를 수집했습니다.
JSON 파일 저장 완료: ./ai_news\ai_times_news_20241121-20241122_p1-1.json

[데이터 수집 정보]
수집 페이지: 1-1 페이지
수집 기간: 2024.11.21 - 2024.11.22
수집 기사: 총 20건

[데이터 미리보기]
                                     title  \
0  법제연구원-구글코리아-개인정보위, 학술대회서 ‘AI 입법 방향성’ 논의   
1             독일, 고속도로에 54GW 태양광 설치 가능성 평가   
2     구글클라우드, 워크스페이스 사이드 패널서 '제미나이' 한국어 지원   
3   플루, 올리브영 '올영라이브'서 25일 라이브 방송서 특별 구성 공개   
4                    켄텍, 영농형태양광협회와 업무협약 체결   

                                         description  \
0  한국법제연구원(원장 한영수)은 서울 강남 파이낸셜센터 21층 대회의실에서 ‘인공지능...   
1  독일 연방도로청은 20일(현지시간) 고속도로 유휴부지 및 관련 시설에 태양광 발전 ...   
2  구글 클라우드가 워크스페이스를 위한 제미나이(Gemini for Google Wor...   
3  지본코스메틱의 대표 브랜드 플루(PLU)는 25일 낮 12시부터 12시30분까지 올...   
4  한국에너지공과대학교(켄텍)은 21일 한국영농형태양광협회와 영농형 태양광의 기술개발 ...   

                                                 url              date  \
0  https://www.aitimes.com/news/articleView.html?...